In [27]:
import pandas as pd 
import os
from itertools import islice

In [28]:
# global variables
min_n, max_n = 5, 8

In [29]:
gen = os.walk('../SymbTr/txt')

paths = [(a, b, c) for a, b, c in gen]
paths = [ (a,c) for a,b,c in paths][0]
paths = [path for path in paths[1]]

makam_dict = {}

for path in paths[1]:
    makam = path.split('--')[0]
    if makam in makam_dict:
        makam_dict[makam] += 1
    else:
        makam_dict[makam] = 1

for m in makam_dict:
    if makam_dict[m] > 19:
        print(m, makam_dict[m])

In [30]:
makam_dict = {'rast', 'ussak'}
paths_new = {}

for makam in makam_dict:
    counter = 0
    paths_list = []
    for path in paths:
        if path.split('--')[0] == makam:
            paths_list.append(path)
            counter += 1
        if counter == 3:
            paths_new[makam] = paths_list
            break
paths_new

{'rast': ['rast--turku--nimsofyan--sira_sira--.txt',
  'rast--sarki--musemmen--vuslatindan_gayri--haci_arif_bey.txt',
  'rast--sarki--senginsemai--sevda_ile_dillendi--munir_nurettin_selcuk.txt'],
 'ussak': ['ussak--yuruksemai--yuruksemai--dila_cunem--mahmut_celebi.txt',
  'ussak--turku--sofyan--telgirafin_tellerine-2.txt',
  'ussak--turku--nimsofyan--ayva_cicek--.txt']}

In [31]:
def get_relative_intervals(list_of_tuples):
    lt = [(0, list_of_tuples[0][1])]
    for i in range(1, len(list_of_tuples)):
        lt.append((list_of_tuples[i][0] - list_of_tuples[i - 1][0], list_of_tuples[i][1]))
    return lt

In [44]:
def get_sublists(listt, num):
    print(listt)
    print(num)
    lists = []
    for i in range(0, len(listt)-num +1):
        lists.append(list(islice(listt, i, i+num)))
    return lists

In [33]:
def separete_lists_from_list_of_tuples(list_of_notes, min_n, max_n):
    #print(list_of_notes)
    komas_list = []
    dur_list = []
    for komas, dur in list_of_notes:
        print(komas)
        print(dur)
        komas_list.append
    
    
    
    
    
    
#inter = []
#dur = []
#for ind, phrase in enumerate(phrases):
#    inter.append([])
#    dur.append([])
#    for n in phrase:
#        inter[ind].append(n[0])
#        dur[ind].append(n[1])

In [34]:
list_of_notes = ((23, (2,2)), (43, (3,3)), (34, (3,4)))
get_similarity_matrix_from_list_of_tuples(list_of_notes, min_n, max_n)

((23, (2, 2)), (43, (3, 3)), (34, (3, 4)))
23
(2, 2)
43
(3, 3)
34
(3, 4)


In [45]:
listt = [23,43,34,54,34,65,67,78]
print(get_sublists(listt, 3))

[23, 43, 34, 54, 34, 65, 67, 78]
3
[[23, 43, 34], [43, 34, 54], [34, 54, 34], [54, 34, 65], [34, 65, 67], [65, 67, 78]]
